In [ ]:
import pandas as pd
import numpy as np 
import plotly.graph_objects as go


In [ ]:
file_path = 'FILE_PATH_PLACEHOLDER'
boxes_data = pd.read_csv(file_path)

In [ ]:
#Convert object data to numeric
for col in boxes_data.columns.tolist():
  if boxes_data[col].dtype == 'object':
    boxes_data[col] = pd.to_numeric(boxes_data[col].astype(str).str.replace(',', '', regex=False), errors='coerce')

# Convert all columns to float, coercing errors
for col in boxes_data.columns:
    boxes_data[col] = pd.to_numeric(boxes_data[col], errors='coerce')

In [ ]:
nom147 = {
    'Ag': np.nan,
    'As': np.nan,
    'Pb': np.nan,
    'Sb': np.nan,
    'Cr': np.nan,
    'Ba': np.nan,
    'Hg': np.nan,
    'Cd': np.nan,
    'Ni': 1600,
    'Tl': 5.2,
    'V': 78 
    
}
nom157 = {
    'Ag': 100,
    'As': 100,
    'Pb': 100,
    'Sb': 10.6,
    'Cr': 100,
    'Ba': 2000,
    'Hg': 4,
    'Cd': 20,
    'Ni': np.nan,
    'Tl': np.nan,
    'V': np.nan 
    
}

In [ ]:
columns = boxes_data.columns.tolist()

In [ ]:
fig = go.Figure()

for col in columns:
    data = boxes_data[col].dropna()
    Q1 = np.percentile(data, 25)
    Q3 = np.percentile(data, 75)
    IQR = Q3 - Q1

    # Asymmetric whiskers
    lower_whisker = max(data.min(), Q1 - 1.5 * IQR)
    upper_whisker = min(data.max(), Q3 + 1.5 * IQR)

    # Only max outlier
    max_outlier = data.max() 

    # Clip data for the box (remove other outliers)
    box_data = np.clip(data, lower_whisker, upper_whisker)

    # Add the box
    fig.add_trace(go.Box(
        y=box_data,
        name=col,
        boxpoints=False,  # hide all individual points
        whiskerwidth=0.7,
        marker_color='lightblue',
        line=dict(color='blue'),
        quartilemethod="linear"
    ))

    fig.add_trace(go.Scatter(
        x=[col],
        y=[nom157[col]],
        mode='markers',
        marker=dict(color='red', size=8),
        marker_symbol='square',
        name='LMP NOM-157-SEMARNAT-2009',
        showlegend=(col == columns[0])
    ))

    fig.add_trace(go.Scatter(
        x=[col],
        y=[nom147[col]],
        mode='markers',
        marker=dict(color='green', size=8),
        marker_symbol='square',
        name='LMP NOM-147-SEMARNAT-2004',
        showlegend=(col == columns[0])  # show legend only once
    ))

    # Add max outlier as a separate scatter point
    if max_outlier is not None:
        fig.add_trace(go.Scatter(
            x=[col],
            y=[max_outlier],
            mode='markers',
            marker=dict(color='red', size=8),
            name='Max outlier',
            showlegend=(col == columns[0])  # show legend only once
        ))

fig.update_layout(
    title="Asymmetric Whiskers with Only Max Outlier",
    yaxis_title="Values",
    xaxis_title="Columns",
    width=1700,
    height=600
)

fig.update_yaxes(type="log")

fig.show()